In [ ]:
# Installing the required libraries
!pip install -r "/content/drive/MyDrive/Masters in AI: Thesis/References/Time-series papers that were read and implemented/Nonstationary-Transfomers/Nonstationary_Transformers-main/Nonstationary_Transformers-main/requirements.txt"

In [ ]:
# Running the non-stationary transformer model for 96, 192, 336 and 720-ahead predictions

import os
import shutil
from google.colab import files

def download_data(model_name):

  pred_lens = [96, 192, 336, 720]

  for pred_len in pred_lens:

    # Create a folder
    folder_name = model_name + '_'+str(pred_len)
    os.makedirs(folder_name, exist_ok=True)

    files_to_download = []

    for i in range(1):
      files_to_download.append(f'metrics_run_{i}_{model_name}_{pred_len}.csv')
      files_to_download.append(f'preds_run_{i}_{model_name}_{pred_len}.csv')
      files_to_download.append(f'trues_run_{i}_{model_name}_{pred_len}.csv')

    files_to_download.append(f'mean_std_metrics_{model_name}_{pred_len}.csv')

    # Move files to the folder
    for file_name in files_to_download:
        shutil.move(file_name, os.path.join(folder_name, file_name))

    # Zip the folder
    shutil.make_archive(folder_name, 'zip', folder_name)

    # Download the zip file
    files.download(f'{folder_name}.zip')
    
bash_command96 = f'"/content/drive/MyDrive/masters back up/Nonstationary-Transfomers/Nonstationary_Transformers-main/Nonstationary_Transformers-main/scripts/Exchange_script/Replication/ns_Transformer96.sh"'
bash_command192 = f'"/content/drive/MyDrive/masters back up/Nonstationary-Transfomers/Nonstationary_Transformers-main/Nonstationary_Transformers-main/scripts/Exchange_script/Replication/ns_Transformer192.sh"'
bash_command336 = f'"/content/drive/MyDrive/masters back up/Nonstationary-Transfomers/Nonstationary_Transformers-main/Nonstationary_Transformers-main/scripts/Exchange_script/Replication/ns_Transformer336.sh"'
bash_command720 = f'"/content/drive/MyDrive/masters back up/Nonstationary-Transfomers/Nonstationary_Transformers-main/Nonstationary_Transformers-main/scripts/Exchange_script/Replication/ns_Transformer720.sh"'

!bash {bash_command96}
!bash {bash_command192}
!bash {bash_command336}
!bash {bash_command720}

download_data('ns_Transformer')

In [ ]:
# Running the non-stationary transformer model for 1-step ahead predictions

import os
import shutil
from google.colab import files

def download_data(model_name):

  pred_len = 1

  targets = ['0', '1', '2', '3', '4', '5', '6', 'OT']

  for target in targets:

    # Create a folder
    folder_name = model_name + '_'+str(pred_len)
    os.makedirs(folder_name, exist_ok=True)

    files_to_download = []

    for i in range(1):
      files_to_download.append(f'metrics_run_{i}_{model_name}_{pred_len}_{target}.csv')
      files_to_download.append(f'preds_run_{i}_{model_name}_{pred_len}_{target}.csv')
      files_to_download.append(f'trues_run_{i}_{model_name}_{pred_len}_{target}.csv')

    #files_to_download.append(f'mean_std_metrics_{model_name}_{pred_len}_{target}.csv')

    # Move files to the folder
    for file_name in files_to_download:
        shutil.move(file_name, os.path.join(folder_name, file_name))

    # Zip the folder
    shutil.make_archive(folder_name, 'zip', folder_name)

    # Download the zip file
    files.download(f'{folder_name}.zip')
    
bash_command1 = f'"/content/drive/MyDrive/masters back up/Nonstationary-Transfomers/Nonstationary_Transformers-main/Nonstationary_Transformers-main/scripts/Exchange_script/Replication/ns_Transformer1.sh"'

!bash {bash_command1}

download_data('ns_Transformer')

In [ ]:
# Running the transformer model with the Monte Carlo Dropout
import numpy as np
import pandas as pd
from scipy import stats
import shutilimport numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
import csv
import shutil
import os
import subprocess

def RSE(pred, true):
    return np.sqrt(np.sum((true - pred) ** 2)) / np.sqrt(np.sum((true - true.mean()) ** 2))


def CORR(pred, true):
    u = ((true - true.mean(0)) * (pred - pred.mean(0))).sum(0)
    d = np.sqrt(((true - true.mean(0)) ** 2 * (pred - pred.mean(0)) ** 2).sum(0))
    return (u / d).mean(-1)


def MAE(pred, true):
    return np.mean(np.abs(pred - true))


def MSE(pred, true):
    return np.mean((pred - true) ** 2)


def RMSE(pred, true):
    return np.sqrt(MSE(pred, true))


def MAPE(pred, true):
    return np.mean(np.abs((pred - true) / true))


def MSPE(pred, true):
    return np.mean(np.square((pred - true) / true))


def metric(pred, true):
    mae = MAE(pred, true)
    mse = MSE(pred, true)
    rmse = RMSE(pred, true)
    mape = MAPE(pred, true)
    mspe = MSPE(pred, true)

    return mae, mse, rmse, mape, mspe

windows = [1]

alphas = [0.01, 0.05, 0.1, 0.15]

output_file_names = list()

for window in windows:

  output_file_name = "output_mc_" + str(window)

  !rm -rf "$output_file_name"

  !mkdir "$output_file_name"

  
  bash_command = f'"/content/drive/MyDrive/masters back up/Nonstationary-Transfomers/Nonstationary_Transformers-main/Nonstationary_Transformers-main/scripts/Exchange_script/ns_Transformer_mc_{window}.sh"'

  !bash {bash_command}

  pred_list = list()

  
  base_path = "/content/results/Exchange_96_5_ns_Transformer_custom_ftMS_sl96_ll48_pl"+str(window)+"_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_h16_l2_0/"
  new_path = output_file_name + "/"
  for i in range(10):
      file_path = f"{base_path}pred_{i}.npy"
      pred = np.load(file_path)
      #files.download(file_path)
      shutil.copy(file_path, new_path)
      pred_list.append(pred)

  pred_arrays = np.array(pred_list)

  array_list = [arr for arr in pred_arrays] # removed [:,-1,0]

  df = pd.DataFrame(array_list).T  # Transpose the DataFrame to have each array as a column

  df.to_csv('mc_raw_output.csv', index=False)
  shutil.move('mc_raw_output.csv', new_path)

  # Calculate minimum, maximum, and mean along axis=1 (equivalent indexes)
  min_values = df.min(axis=1)
  max_values = df.max(axis=1)
  mean_values = df.mean(axis=1)

  # Convert the results back to arrays if needed
  min_array = np.array(min_values).reshape(-1, 1)
  max_array = np.array(max_values).reshape(-1, 1)
  mean_array = np.array(mean_values).reshape(-1, 1)

  # Create a DataFrame with named columns
  min_max_mean = {
      'min': min_array,
      'max': max_array,
      'mean': mean_array
  }

  # Flatten the arrays
  for key, value in min_max_mean.items():
      min_max_mean[key] = value.flatten()

  df = pd.DataFrame(min_max_mean)

  # Save the DataFrame to a CSV file
  df.to_csv('min_max_mean.csv', index=False)
  shutil.move('min_max_mean.csv', new_path)

  #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> calculating alpha

  # Define a function to calculate lower, upper, and mean for a given alpha for each row
  def calculate_confidence_intervals_for_row(row, alpha):
      mean = np.mean(row)
      std_dev = np.std(row)
      n = len(row)
      sem = std_dev / np.sqrt(n)
      critical_value = stats.norm.ppf(1 - alpha / 2)  # For two-tailed confidence interval
      margin_of_error = critical_value * sem
      lower_bound = mean - margin_of_error
      upper_bound = mean + margin_of_error
      return lower_bound, upper_bound, mean

  # Loop over different confidence levels (alphas)
  for alpha in alphas:
      # Calculate confidence intervals for each alpha for each row
      confidence_intervals = df.apply(lambda row: calculate_confidence_intervals_for_row(row, alpha), axis=1, result_type='expand')
      confidence_intervals.columns = ['lower', 'upper', 'mean']

      # Save DataFrame to a CSV file
      filename = f"min_max_mean_alpha_{alpha}.csv"
      confidence_intervals.to_csv(filename, index=False)
      shutil.move(filename, new_path)

  
  # Create the plot
  plt.figure(figsize=(10, 6))

  true_0 = np.load(f"{base_path}true_0.npy")
  #files.download(f"{base_path}true_0.npy")
  shutil.move(f"{base_path}true_0.npy", new_path)

  x_test = np.load(f"{base_path}x_test.npy")
  shutil.move(f"{base_path}x_test.npy", new_path)

  checkpoint_base_path = "/content/checkpoints/Exchange_96_5_ns_Transformer_custom_ftMS_sl96_ll48_pl1_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_h16_l2_0/"
  shutil.copy(f"{checkpoint_base_path}checkpoint.pth", new_path)

  true_last = true_0#[:, -1, 0]

  # Plot the mean values as a line
  plt.plot(true_last[:100], label='Actual', color='blue')
  plt.plot(mean_array[:100], label='Mean', color='black')

  # Fill the area between the minimum and maximum values to represent confidence bands
  plt.fill_between(range(len(min_array[:100])), min_array[:100].squeeze(), max_array[:100].squeeze(), alpha=0.3, color='gray', label='Confidence Band')

  # Customize the plot
  if window == 1:
    plt.title(f"MC Dropout with Model Forecasting {window} Day Ahead")
  else:
    plt.title(f"MC Dropout with Model Forecasting {window} Days Ahead")
  plt.xlabel('Index')
  plt.ylabel('Values')
  plt.legend()
  plt.grid(True)


  plt.savefig("plot.png")
  #files.download("plot.png")
  shutil.move("plot.png", new_path)

  # Show the plot
  plt.show()

  metrics = metric(mean_array, true_last)


  # Specify the CSV file path
  csv_file = "metrics.csv"  # You can choose the desired file name


  # Create a header
  header = ["mae", "mse", "rmse", "mape", "mspe"]

  # Write the data to the CSV file
  with open(csv_file, 'w', newline='') as file:
      writer = csv.writer(file)

      # Write the headers
      writer.writerow(header)

      # Write the data as a single row
      writer.writerow(metrics)

  #files.download("metrics.csv")
  shutil.move("metrics.csv", new_path)

  !zip -r "$output_file_name" "$output_file_name"

  output_file_names.append(output_file_name)

  !rm -rf checkpoints
  !rm -rf results
  !rm -rf test_results
  !rm -rf metrics.csv
  !rm -rf plot.png
  !rm -rf results.txt
  !rm -rf result.txt
  !rm -rf min_max_mean.csv

for output_file_name in output_file_names:
  files.download(output_file_name + ".zip")

In [ ]:
# Transformer with the EnbPI uncertainty
import os
import shutil

!pip install puncc

bash_command = f'"/content/drive/MyDrive/masters back up/Nonstationary-Transfomers/Nonstationary_Transformers-main/Nonstationary_Transformers-main/scripts/Exchange_script/ns_Transformer_EnbPI.sh"'

!bash {bash_command}

# Downloading the models and residuals

alphas = [0.01, 0.05, 0.1, 0.15]
ss = [100, 50, 25, 20, 10, 5]

for alpha in alphas:
  for s in ss:

    # Create a folder
    folder_name = 'transformer_confidence_run_data'+str(int((1-alpha)*100))+'_s_'+str(s)
    os.makedirs(folder_name, exist_ok=True)

    files_to_download = ['plot_with_CP_PUNCC_alp_'+str(int((1-alpha)*100))+'_s_'+str(s)+'.png', 'BootstrappedPredictionsvsActual.png','transformer_confidence_metrics_alp_'+str(int((1-alpha)*100))+'_s_'+str(s)+'.csv','y_true_and_pred_alp_'+str(int((1-alpha)*100))+'_s_'+str(s)+'.csv']

    # Move files to the folder
    for file_name in files_to_download:
        shutil.move(file_name, os.path.join(folder_name, file_name))

    # Zip the folder
    shutil.make_archive(folder_name, 'zip', folder_name)

    # Download the zip file
    files.download(f'{folder_name}.zip')

alpha = 0.05
s = 100

# Create a folder
folder_name = 'transformer_confidence_run_model_data'
os.makedirs(folder_name, exist_ok=True)

files_to_download = []

# Download files named 0.pth, 1.pth... up until 29.pth
for i in range(30):
    files_to_download.append(f'{i}.pth')

# Download files whose name begins with 'plot'
files_to_download += [file_name for file_name in os.listdir() if file_name.startswith('plot')]

# Download files whose name is 'result.txt'
files_to_download.append('result.txt')

# Move files to the folder
for file_name in files_to_download:
    shutil.move(file_name, os.path.join(folder_name, file_name))

# Zip the folder
shutil.make_archive(folder_name, 'zip', folder_name)

# Download the zip file
files.download(f'{folder_name}.zip')


In [ ]:
!pip install arch
!pip install puncc

bash_command = f'"/content/drive/MyDrive/masters back up/Nonstationary-Transfomers/Nonstationary_Transformers-main/Nonstationary_Transformers-main/scripts/Exchange_script/ns_Transformer_GARCH.sh"'

!bash {bash_command}

import os
import shutil
from google.colab import files

alphas = [0.01, 0.05, 0.1, 0.15]
targets=['0', '1', '2', '3', '4', '5', '6', 'OT']
lags = str(8)

for target in targets:

  for alpha in alphas:

    # Create a folder
    folder_name = 'garch_transformer_confidence_run_data_alpha_'+str(alpha)+'_target_' + target +'_lags_' + lags
    os.makedirs(folder_name, exist_ok=True)

    files_to_download = ['plot_with_CP_GARCH_alpha_'+str((1-alpha)*100)+'_'+target+'.png', 'garch_transformer_confidence_metrics_alp_'+str(int((1-alpha)*100))+'_'+target+'.csv','garch_transforemer_y_true_and_pred_alp_'+str(int((1-alpha)*100))+'_'+target+'.csv']

    # Move files to the folder
    for file_name in files_to_download:
        shutil.move(file_name, os.path.join(folder_name, file_name))

    # Zip the folder
    shutil.make_archive(folder_name, 'zip', folder_name)

    # Download the zip file
    files.download(f'{folder_name}.zip')

  for alpha in alphas:

    # Create a folder
    folder_name = 'val_garch_transformer_confidence_run_data_alpha_'+str(alpha)+'_target_' + target +'_lags_' + lags
    os.makedirs(folder_name, exist_ok=True)

    files_to_download = ['val_plot_with_CP_GARCH_alpha_'+str((1-alpha)*100)+'_'+target+'.png', 'val_garch_transformer_confidence_metrics_alp_'+str(int((1-alpha)*100))+'_'+target+'.csv','val_garch_transforemer_y_true_and_pred_alp_'+str(int((1-alpha)*100))+'_'+target+'.csv']

    # Move files to the folder
    for file_name in files_to_download:
        shutil.move(file_name, os.path.join(folder_name, file_name))

    # Zip the folder
    shutil.make_archive(folder_name, 'zip', folder_name)

    # Download the zip file
    files.download(f'{folder_name}.zip')
    


for target in targets:

  for alpha in alphas:

    # Create a folder
    folder_name = 'garch_transformer_confidence_run_data_alpha_'+str(alpha)+'_target_' + target +'_lags_' + lags

    # Download the zip file
    files.download(f'{folder_name}.zip')

  for alpha in alphas:

    # Create a folder
    folder_name = 'val_garch_transformer_confidence_run_data_alpha_'+str(alpha)+'_target_' + target +'_lags_' + lags
    # Download the zip file
    files.download(f'{folder_name}.zip')